In [1]:
!pip install chromadb langchain-upstage
!python -m spacy download ko_core_news_sm pymupdf
!pip install -U langchain-community
!pip install langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.8 MB/s eta 0:00:

### 벡터 디비에 저장

In [2]:
from langchain.document_loaders import PyMuPDFLoader
from langchain_upstage import UpstageEmbeddings
from langchain.text_splitter import SpacyTextSplitter
from langchain_chroma import Chroma

documents = PyMuPDFLoader("가뭄 대비 농작물 관리 요령.pdf").load()

# 언어 모델을 사용해서 분할
text_splitter = SpacyTextSplitter(
    chunk_size = 300, # 분할할 크기
    chunk_overlap = 50, # 분할 할 때 겹치는 크기
    pipeline = 'ko_core_news_sm'
)

splitted_documents = text_splitter.split_documents(documents)
print(len(splitted_documents))

42


In [3]:
embeddings = UpstageEmbeddings(model = 'embedding-query',
                               api_key = '<API_KEY>')
vectorstore = Chroma.from_documents(documents=documents,
                                    embedding=embeddings,
                                    collection_name='farm', # 저장할 documents의 별명
                                    persist_directory='./vectorDB') # 저장할 경로

vectorstore.add_documents(splitted_documents)

['d9b77b37-d0e3-4535-9630-c9852e27f180',
 'ee9606bd-c2e1-4791-bbee-c27c1a5e9670',
 'b3daca3d-9b3c-44b2-a58b-fa61b1bbdd75',
 'fa693a3e-1f91-4139-b2a2-1900e4bcbb41',
 '5888a220-5e4c-4007-aa5f-d22e9e0ec56c',
 'e5fc316a-795d-486a-a50d-abae348db4e7',
 '6721a15a-c2cf-4e09-8839-06f2ba45cd15',
 '299e746a-c3a2-4239-85f2-51feba350364',
 '353956a7-6dd9-4f8c-a32c-703351fba990',
 '4314eef2-64c1-4797-8652-b46349d26c84',
 '49817894-bb3d-435e-a007-30d30edc54a8',
 '79c77ef8-294a-4e55-a7df-ff273f88072e',
 '9ffb0ae9-315b-4fee-98c7-6495b87067d3',
 '0a106550-155c-4030-b70d-9f53f3bddc40',
 '809d014e-a032-4c0b-8fcd-f31f4431a10d',
 '407502b7-0df7-4069-8c2b-52dc3ab96f70',
 '1ed919df-bb26-4e3e-a34f-a773aa559e00',
 '8a1a1c8e-e36b-433e-be63-ea99924a5176',
 '44c2e348-f6c1-4bb5-b45b-e4b8c9d7148d',
 'd7cb815f-4c11-4dfa-afa0-51121d6433da',
 '8f06abe6-6dc4-44cd-b37f-b09926690902',
 'a452a1f7-6c5f-4726-99fd-370f08515ce7',
 'bc178d83-361f-44fa-a299-d25f5281b78c',
 'e14ec47a-2cf6-4067-bd2d-cce74445c186',
 'c3111889-bb80-

In [4]:
query = "월동무 가뭄발생시 조치사항을 알려주세요."
vectorstore_documents = vectorstore.similarity_search(query)

print(len(vectorstore_documents)) # 검색 된 비슷한 문서
for document in vectorstore_documents:
    print(document)

4
page_content='- 13 -
 m 사후대책
   - 토양의 수분 증발을 줄이기 위해 표토를 긁어 주어 모세관을 끊어 줌
   - 가뭄이 끝난 후 생육회복을 위해 물 공급은 물론 요소 0.3%액을 엽면에 살
포하여 생육을 촉진
   - 정식 직후 가뭄에 의해 일부 양배추가 고사할 경우 보식을 하고 정식시기
가 지난 경우는 대체작물을 식재
사.' metadata={'author': 'USER', 'creationDate': "D:20231101091957+09'00'", 'creator': 'Hwp 2018 10.0.0.13462', 'file_path': '가뭄 대비 농작물 관리 요령.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20231101091957+09'00'", 'page': 12, 'producer': 'Hancom PDF 1.3.0.542', 'source': '가뭄 대비 농작물 관리 요령.pdf', 'subject': '', 'title': '', 'total_pages': 16, 'trapped': ''}
page_content='- 12 -
바.

양배추
q 피해발생 여건
 m 양배추는 90%가 수분으로 구성되어 있어 토양으로부터의 수분공급이 매우 
중요.' metadata={'author': 'USER', 'creationDate': "D:20231101091957+09'00'", 'creator': 'Hwp 2018 10.0.0.13462', 'file_path': '가뭄 대비 농작물 관리 요령.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20231101091957+09'00'", 'page': 11, 'producer': 'Hancom PDF 1.3.0.542', 'source': '가뭄 대비 농작물 관리 요령.pdf', 'subject': '', 'title': '', 'total_pages': 16, 'trap

### 벡터 디비 참고해서 답변

In [5]:
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate
from langchain_chroma import Chroma
from langchain_upstage import ChatUpstage

load_vectorstore = Chroma(collection_name='farm',
                         embedding_function=embeddings,
                         persist_directory='./vectorDB')

query = "월동무 가뭄발생시 조치사항에 대해서 알려주세요."
vectorstore_documents = load_vectorstore.similarity_search(query)

documents_string = ""

for idx, document in enumerate(vectorstore_documents):
    documents_string += f"""

    --- {idx+1} 번째 문서 ---
    {document.page_content}
    """

prompt = PromptTemplate(
    template = """ 아래 문서를 바탕으로 질문에 답변해주세요.

    문장 : {document}

    질문 : {query}
    """,
    input_variables = ['document', 'query']
)

final_prompt = prompt.format(document = documents_string, query = query)

llm = ChatUpstage(model = 'solar-pro',
                  api_key = '<API_KEY>')
result = llm.invoke([HumanMessage(content = final_prompt)])
print(result.content)

월동무 가뭄 발생 시 조치사항은 다음과 같습니다:

사전 대책:
- 월동무 파종 후 토양이 건조해지면 반드시 관수해야 하므로 관수시설을 사전 설치합니다.
- 겨울 동안 건조해지기 쉬우므로 멀칭하여 재배합니다.
- 잎끝 마름 현상(엽선단 고사) 대책으로 1~2회 관수를 실시하여 영양을 공급합니다.
- 잎끝 마름 현상이 자주 발생하는 포장은 퇴비를 10a당 1,500~2,500kg 시용하고, 토양이 산성인 포장은 석회 120~150kg/10a 살포하여 파종합니다.

사후 대책:
- 가뭄 발생 시, 토양의 수분 증발을 줄이기 위해 표토를 긁어 주어 모세관을 끊어 줍니다.
- 가뭄이 끝난 후 생육회복을 위해 물 공급은 물론 요소 0.3%액을 엽면에 살포하여 생육을 촉진합니다.
- 정식 직후 가뭄에 의해 일부 월동무가 고사할 경우 보식을 하고, 정식시기가 지난 경우는 대체작물을 식재합니다.


### RetrievalQA 활용하기

In [6]:
from langchain_chroma import Chroma
from langchain_upstage import ChatUpstage
from langchain_upstage import UpstageEmbeddings
from langchain.chains import RetrievalQA

UPSTAGE_API_KEY = '<API_KEY>'
llm = ChatUpstage(model = 'solar-pro',
                  api_key = UPSTAGE_API_KEY)
embeddings = UpstageEmbeddings(model = 'embedding-query',
                               api_key = UPSTAGE_API_KEY)
load_vectorstore = Chroma(collection_name='farm',
                         embedding_function=embeddings,
                         persist_directory='./vectorDB')

retriever = load_vectorstore.as_retriever()

qa_retriever = RetrievalQA.from_llm(
    llm = llm,
    retriever = retriever,
    return_source_documents = True # 응답에 원본 문서 포함 여부
  )

query = '월동무 가뭄발생시 조치사항'
result = qa_retriever.invoke(query)
print(result['result'])

월동무 가뭄 발생 시 사전대책으로는 파종 직후, 월동 전, 웃거름 시비할 때, 구 비대가 
진행되는 5월 이후 수확 전까지 토양이 건조하면 반드시 관수해야 하므로 관수시설 
사전 설치, 겨울 동안 건조해지기 쉬우므로 멀칭하여 재배하는 것이 좋습니다. 또한 
잎끝 마름 현상(엽선단 고사) 대책으로 1~2회 관수를 실시하여 영양 공급을 해야 합
니다. 구체적인 관리 방법으로는 퇴비를 10a당 1,500~2,500kg 사용하고, 토양이 산
성인 포장은 석회 120~150kg/10a 살포하여 파종할 수 있습니다. 사후대책은 토양
의 수분 증발을 줄이기 위해 표토를 긁어 주어 모세관을 끊어 주고, 가뭄이 끝난 후 
생육회복을 위해 물 공급은 물론 요소 0.3%액을 엽면에 살포하여 생육을 촉진시키
는 것이 좋습니다.
